In [1]:
import cv2
from cvzone.HandTrackingModule import HandDetector
from time import sleep
import numpy as np
import cvzone
from pynput.keyboard import Controller

In [10]:
cap=cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)

detector=HandDetector(detectionCon=0.8)
finalText=""

keyboard=Controller()

keys=[["Q","W","E","R","T","Y","U","I","O","P"],["A","S","D","F","G","H","J","K","L",";"],["Z","X","C","V","B","N","M",",",".","/"]]
#handle the drawing button
 
# def drawall(img,button_list):
#     for button in button_list:
#         x,y=button.pos
#         w,h=button.size
#         cv2.rectangle(img,button.pos,(  x+w,y+h),(255,0,255),cv2.FILLED)
#         cv2.putText(img,button.text,(x+15,y+65),cv2.FONT_HERSHEY_PLAIN,4,(255,255,255),4)
#     return img




def drawall(img, buttonList):
    imgNew = np.zeros_like(img, np.uint8)
    for button in buttonList:
        x, y = button.pos
        cvzone.cornerRect(imgNew, (button.pos[0], button.pos[1], button.size[0], button.size[1]),
                          20, rt=0)
        cv2.rectangle(imgNew, button.pos, (x + button.size[0], y + button.size[1]),
                      (255, 0, 255), cv2.FILLED)
        cv2.putText(imgNew, button.text, (x + 40, y + 60),
                    cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 3)

    out = img.copy()
    alpha = 0.3
    mask = imgNew.astype(bool)
    print(mask.shape)
    out[mask] = cv2.addWeighted(img, alpha, imgNew, 1 - alpha, 0)[mask]
    return out






class Button():
    def __init__(self,pos,text,size=[85,85]):
        self.pos=pos
        self.size=size
        self.text=text
        

button_list=[]  
for i in range (len(keys)):
        for j,key in enumerate(keys[i]):
            button_list.append(Button([j*100+50,100*i+50],key))

while True:
    success,img=cap.read()

    img=detector.findHands(img)
    lmlist,bboxinfo=detector.findPosition(img)
    img=drawall(img,button_list)
    #checking hand is near
    if lmlist:
        for button in button_list:
            x,y=button.pos
            w,h=button.size
            # landmark 8 for point and 12 for click can see pipeline
            if x<lmlist[8][0]<x+w and y<lmlist[8][1]<y+h:
                cv2.rectangle(img,button.pos,(x+w,y+h),(175,0,175),cv2.FILLED)
                cv2.putText(img,button.text,(x+15,y+65),cv2.FONT_HERSHEY_PLAIN,4,(255,255,255),4)
                l, _, _=detector.findDistance(8,12,img,draw=False)
                print(l)

                # when we click
                if l<40:
                    keyboard.press(button.text)
                    cv2.rectangle(img,button.pos,(x+w,y+h),(0,255,0),cv2.FILLED)
                    cv2.putText(img,button.text,(x+15,y+65),cv2.FONT_HERSHEY_PLAIN,4,(255,255,255),4)
                    finalText+=button.text
                    sleep(0.5)


    cv2.rectangle(img,(50,350),(700,450),(175,0,175),cv2.FILLED)
    cv2.putText(img,finalText,(60,425),cv2.FONT_HERSHEY_PLAIN,5,(255,255,255),5)


    
     
    cv2.imshow("Image",img)
    
    key=cv2.waitKey(1)
    if key==ord('q'):
        break